In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np

In [ ]:
#fungsi untuk mengekstrack judul produk
# jika tidak ditemukan judul produk maka akan menampilkan string kosong, jika dtemukan judul maka akan mengembalikan nilai judul produk
def get_title(soup):
  try:
    #Outer tag objek
    title = soup.find("span", attrs={'id' : 'productTitle'})
    
    #Inner Navigatable String Objek
    title_value = title.text
    
    #title sebagai nilai string
    title_string = title_value.strip()
    
  except AttributeError:
    title_string = " "
    
  return title_string

# fungsi untuk mengekstrack harga produk
def get_price(soup):
    
    try:
      price = soup.find("span", attrs={'class': 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).find("span", attrs={'class' : 'a-offscreen'}).string.strip()
      
    except AttributeError:
    
        price = ""
        
    return price

# fungsi untuk mengekstrack rating produk
def get_rating(soup):
    
    try:
      rating = soup.find("i", attrs={'class': 'a-icon a-icon-star a-star-4-5'}).string.strip()
      
    except AttributeError:
      
      try:
        rating = soup.find("span", attrs={'class': 'a-icon-alt'}).string.strip()
        
      except:
        rating = ""
        
    return rating

# fungsi untuk mengekstrack reviews
def get_review_count(soup):
    
    try:
      review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'}).string.strip()
      
    except AttributeError:
      
      review_count = ""
        
    return review_count

# fungsi untuk mengekstrack status ketersediaan
def get_availability(soup):
    
    try:
      available = soup.find("div", attrs={'id': 'availability'})
      available = available.find("span").string.strip()
      
    except AttributeError:
      available = "Not Available"
        
    return available

In [3]:
if __name__ == '__main__':
  
  #HEADERS for REQUESTS
  HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36', 'Accept-Language' : 'en-US, en;q=0.5'})
  
  #webpage URL
  URL = "https://www.amazon.in/s?k=laptop+gaming&crid=3G0LLX8E96V62&sprefix=laptop+gamin%2Caps%2C380&ref=nb_sb_noss_2"
  
  #HTTP request
  webpage = requests.get(URL, headers = HEADERS)
  
  #mengubah menjadi format yang benar
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  links = soup.find_all("a", attrs={'class' : 'a-link-normal s-no-outline'})
  
  links_list = []
  
  for link in links:
          links_list.append(link.get('href'))
  
  d = {"title" :[], "price" :[], "rating" :[], "reviews" :[], "availability" :[]}
  
  for link in links_list:
    new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)
    
    new_soup = BeautifulSoup(new_webpage.content, 'html.parser')
    
    d['title'].append(get_title(new_soup))
    d['price'].append(get_price(new_soup))
    d['rating'].append(get_rating(new_soup))
    d['reviews'].append(get_review_count(new_soup))
    d['availability'].append(get_availability(new_soup))
    
amazon_df = pd.DataFrame.from_dict(d)
amazon_df['title'].replace('', np.nan, inplace=True)
amazon_df = amazon_df.dropna(subset=['title'])
amazon_df.to_csv("amazon_data.csv", header=True, index=True)


In [19]:
amazon_df

,title,price,rating,reviews,availability
0,"HP Victus Gaming Latest AMD Ryzen 5 5600H Processor 16.1 inch(40.9 cm) FHD Gaming Laptop (8GB RAM/512GB SSD/4GB Radeon RX5500M Graphics/B&O/Backlit KB/Win 10/MS Office/Xbox Game Pass),16-E0162ax",Rp,4.3 out of 5 stars,"1,285 ratings",In stock
1,"HP Victus Gaming Laptop 11th Gen Intel Core i5-11400H16.1 inch(40.9 cm) FHD IPS Gaming Laptop(16GB RAM/512GB SSD/NVIDIA GeForce GTX 1650 graphics/144Hz/Backlit KB/Wi11/MSO/B&O/Alexa),16-d0311TX",Rp,5.0 out of 5 stars,7 ratings,In stock
2,"ASUS TUF Gaming A15, 15.6-inch (39.62 cms) FHD 144Hz, AMD Ryzen 5 4600H, 4GB NVIDIA GeForce GTX 1650, Gaming Laptop (8GB/512GB SSD/Windows 11/Black/2.3 Kg), FA506IHRZ-HN111W",Rp,4.3 out of 5 stars,185 ratings,In stock
3,"ASUS ROG Strix G17, 17.3-inch (43.94 cms) FHD 144Hz, AMD Ryzen 7 4800H, RTX 3050 Ti 4GB Graphics, Gaming Laptop (16GB/1TB SSD/Windows 11//Gray/2.4 kg), G713IE-HX040W",Rp,4.5 out of 5 stars,111 ratings,In stock
4,"MSI Katana GF76, Intel 11th Gen. i7-11800H, 43CM FHD 144Hz Gaming Laptop (16GB/512GB NVMe SSD/Windows 11 Home/Nvidia RTX3050 4GB GDDR6/Black/2.6Kg), 11UC-848IN",Rp,4.1 out of 5 stars,59 ratings,In stock
5,HP Victus Gaming Latest AMD Ryzen 7-5800H 16.1 inch(40.9 cm) FHD Gaming Laptop (16GB RAM/ 512GB SSD/4Gb RTX 3050 Graphics/Flicker Free/144Hz Display/B&O/Backlit Kb/Win 11/MSO/Xbox Pass) 16-e0351ax,Rp,4.3 out of 5 stars,259 ratings,In stock
6,"HP Victus Gaming Latest AMD Ryzen 5 5600H Processor 16.1 inch(40.9 cm) FHD Gaming Laptop (8GB RAM/512GB SSD/4GB Radeon RX5500M Graphics/B&O/Backlit KB/Win 10/MS Office/Xbox Game Pass),16-E0162ax",Rp,4.3 out of 5 stars,"1,285 ratings",In stock
7,"HP Victus Gaming Latest 12th Gen Intel Core i5 12450H Processor 15.6 inch(39.6 cm) FHD Gaming Laptop (8GB RAM/512GB SSD/GTX 1650 4GB Graphics/144Hz/BL KB/Win 11/MSO/B&O/Alexa/Xbox Pass), 15-fa0070TX",Rp,4.1 out of 5 stars,371 ratings,In stock
8,"Lenovo IdeaPad Gaming 3 AMD Ryzen 7 5800H 15.6"" (39.62cm) FHD IPS 120Hz Gaming Laptop (8GB/512GB SSD/Win 11/Office 2021/NVIDIA RTX 3050 4GB/Alexa/3 Month Game Pass/Shadow Black/2.2Kg), 82K201V2IN",Rp,4.4 out of 5 stars,166 ratings,In stock
9,"ASUS TUF Dash F15 (2022), 15.6"" (39.62 cms) FHD 144Hz, Intel Core i5-12450H 12th Gen, 4GB GeForce RTX 3050 Graphics, Gaming Laptop (16GB/512GB SSD/Windows 11/with Office/Black/2 kg), FX517ZC-HN084WS",Rp,4.3 out of 5 stars,59 ratings,In stock


In [18]:
amazon_df

,title,price,rating,reviews,availability
0,"HP Victus Gaming Latest AMD Ryzen 5 5600H Processor 16.1 inch(40.9 cm) FHD Gaming Laptop (8GB RAM/512GB SSD/4GB Radeon RX5500M Graphics/B&O/Backlit KB/Win 10/MS Office/Xbox Game Pass),16-E0162ax",Rp,4.3 out of 5 stars,"1,285 ratings",In stock
1,"HP Victus Gaming Laptop 11th Gen Intel Core i5-11400H16.1 inch(40.9 cm) FHD IPS Gaming Laptop(16GB RAM/512GB SSD/NVIDIA GeForce GTX 1650 graphics/144Hz/Backlit KB/Wi11/MSO/B&O/Alexa),16-d0311TX",Rp,5.0 out of 5 stars,7 ratings,In stock
2,"ASUS TUF Gaming A15, 15.6-inch (39.62 cms) FHD 144Hz, AMD Ryzen 5 4600H, 4GB NVIDIA GeForce GTX 1650, Gaming Laptop (8GB/512GB SSD/Windows 11/Black/2.3 Kg), FA506IHRZ-HN111W",Rp,4.3 out of 5 stars,185 ratings,In stock
3,"ASUS ROG Strix G17, 17.3-inch (43.94 cms) FHD 144Hz, AMD Ryzen 7 4800H, RTX 3050 Ti 4GB Graphics, Gaming Laptop (16GB/1TB SSD/Windows 11//Gray/2.4 kg), G713IE-HX040W",Rp,4.5 out of 5 stars,111 ratings,In stock
4,"MSI Katana GF76, Intel 11th Gen. i7-11800H, 43CM FHD 144Hz Gaming Laptop (16GB/512GB NVMe SSD/Windows 11 Home/Nvidia RTX3050 4GB GDDR6/Black/2.6Kg), 11UC-848IN",Rp,4.1 out of 5 stars,59 ratings,In stock
5,HP Victus Gaming Latest AMD Ryzen 7-5800H 16.1 inch(40.9 cm) FHD Gaming Laptop (16GB RAM/ 512GB SSD/4Gb RTX 3050 Graphics/Flicker Free/144Hz Display/B&O/Backlit Kb/Win 11/MSO/Xbox Pass) 16-e0351ax,Rp,4.3 out of 5 stars,259 ratings,In stock
6,"HP Victus Gaming Latest AMD Ryzen 5 5600H Processor 16.1 inch(40.9 cm) FHD Gaming Laptop (8GB RAM/512GB SSD/4GB Radeon RX5500M Graphics/B&O/Backlit KB/Win 10/MS Office/Xbox Game Pass),16-E0162ax",Rp,4.3 out of 5 stars,"1,285 ratings",In stock
7,"HP Victus Gaming Latest 12th Gen Intel Core i5 12450H Processor 15.6 inch(39.6 cm) FHD Gaming Laptop (8GB RAM/512GB SSD/GTX 1650 4GB Graphics/144Hz/BL KB/Win 11/MSO/B&O/Alexa/Xbox Pass), 15-fa0070TX",Rp,4.1 out of 5 stars,371 ratings,In stock
8,"Lenovo IdeaPad Gaming 3 AMD Ryzen 7 5800H 15.6"" (39.62cm) FHD IPS 120Hz Gaming Laptop (8GB/512GB SSD/Win 11/Office 2021/NVIDIA RTX 3050 4GB/Alexa/3 Month Game Pass/Shadow Black/2.2Kg), 82K201V2IN",Rp,4.4 out of 5 stars,166 ratings,In stock
9,"ASUS TUF Dash F15 (2022), 15.6"" (39.62 cms) FHD 144Hz, Intel Core i5-12450H 12th Gen, 4GB GeForce RTX 3050 Graphics, Gaming Laptop (16GB/512GB SSD/Windows 11/with Office/Black/2 kg), FX517ZC-HN084WS",Rp,4.3 out of 5 stars,59 ratings,In stock
